In [12]:
# Import myfunc at cix folder

import sys
sys.path.insert(0, '../cix')

import myfuncs as mf
#from myfuncs import taylor_butina_cluster

from myfuncs import smidf2arena

import scipy.spatial.distance as ssd

from chemfp import search

import time

In [2]:
smis = mf.smif2smis('./train300000.smi')
#smis = mf.corrsmis(smis)[2]
smidft = mf.smis2smidf(smis)
del smis

In [3]:
arena = smidf2arena(smidft)

In [4]:
cl = mf.clusmidf(smidft, arena = arena)

Clustering time: 00:00:50


In [8]:
len(cl)

194526

In [24]:
# Diversity analysis 
#d = mf.distance_matrix(arena)

similarity_table = search.threshold_tanimoto_search_symmetric(arena, threshold = 0.8)

In [26]:
ct = sorted(((len(indices), i, indices)
                                 for (i,indices) in enumerate(similarity_table.iter_indices())),
                            reverse=True)

In [ ]:
clr = mf.taylor_butina_cluster(similarity_table)

In [47]:
len(clr.clusters)

33271

In [49]:
cs = clr.clusters

In [50]:
out = []
        for i in range(len(clus_res.clusters)):
            cl = []
            c = clus_res.clusters[i]
            cl.append(arena.ids[c[0]])
            cl.extend([arena.ids[x] for x in c[1]])
            out.append(cl)
        for i in range(len(clus_res.false_singletons)):
            cl = [arena.ids[clus_res.false_singletons[i]]]
            out.append(cl)
        for i in range(len(clus_res.true_singletons)):
            cl = [arena.ids[clus_res.true_singletons[i]]]
            out.append(cl)
        

list

In [65]:
cs_sorted = sorted([(len(c[1]), cl[1], c[0]) for c in cs], reverse = True)

In [67]:
[c[0] for c in cs_sorted]

[888,
 494,
 453,
 381,
 311,
 250,
 245,
 234,
 231,
 214,
 204,
 194,
 184,
 179,
 172,
 170,
 156,
 152,
 151,
 147,
 141,
 133,
 122,
 122,
 120,
 112,
 110,
 109,
 107,
 106,
 104,
 101,
 98,
 94,
 90,
 89,
 88,
 85,
 84,
 84,
 82,
 80,
 80,
 79,
 79,
 79,
 78,
 78,
 76,
 75,
 75,
 74,
 73,
 72,
 72,
 71,
 71,
 68,
 67,
 67,
 66,
 66,
 66,
 66,
 66,
 65,
 64,
 63,
 63,
 62,
 62,
 62,
 62,
 62,
 61,
 61,
 59,
 59,
 59,
 59,
 59,
 58,
 58,
 57,
 57,
 57,
 57,
 56,
 56,
 55,
 55,
 55,
 54,
 54,
 54,
 54,
 54,
 54,
 53,
 53,
 53,
 52,
 52,
 52,
 52,
 51,
 51,
 51,
 51,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 49,
 49,
 49,
 49,
 49,
 49,
 48,
 48,
 47,
 46,
 46,
 46,
 45,
 45,
 45,
 44,
 44,
 44,
 44,
 43,
 43,
 43,
 43,
 43,
 43,
 43,
 42,
 42,
 42,
 42,
 42,
 42,
 42,
 42,
 41,
 41,
 41,
 41,
 41,
 40,
 40,
 40,
 40,
 40,
 40,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 39,
 38,
 38,
 38,
 38,
 38,
 38,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 37,
 36,
 36,
 36,
 36,
 36,
 35,
 35,
 35,
 35,
 35

In [73]:
c = cs_sorted[0]

In [77]:
c[2]

170973

In [83]:
def clusmidf(smidf, th = 0.8, method = 'butina', arena = None):
    
    if method != 'butina' and method != 'cl':
        print('Please select butina or cl')
        return None
        
    # Init time counter
    start = time.time()
    
    # Get the arena
    if arena is None:
        arena = smidf2arena(smidf)

    # Do the clustering
    if method == 'butina':
        # Generate the similarity table
        similarity_table = search.threshold_tanimoto_search_symmetric(arena, threshold = th)
    
        # Cluster the data
        clus_res = taylor_butina_cluster(similarity_table)
        
        # Output
        out = []
        # We need to re-sort the clusters as the creation of them does not generate a monotonously decreasing list
        cs_sorted = sorted([(len(c[1]), c[0]) for c in clus_res.clusters], reverse = True)
        for i in range(len(cs_sorted)):
            cl = []
            c = cs_sorted[i]
            cl.append(arena.ids[c[2]]) # Retrieve the arenaid of the centroid and add to the cluster
            cl.extend([arena.ids[x] for x in c[1]]) # Retrieve the arenaid of the neighbors and add to cluster
            out.append(cl)
        for i in range(len(clus_res.false_singletons)):
            cl = [arena.ids[clus_res.false_singletons[i]]]
            out.append(cl)
        for i in range(len(clus_res.true_singletons)):
            cl = [arena.ids[clus_res.true_singletons[i]]]
            out.append(cl)
        
    elif method == 'cl':
        # Generate the condensed distance table
        distances  = ssd.squareform(distance_matrix(arena))
        
        # Cluster the data
        clus_res = fcluster(linkage(distances, method='complete'), th, 'distance')
        
        # Ouptut
        aids = arena.ids
        out = []
        for i in np.unique(clus_res):
            cl = [aids[i] for i in list(np.where(clus_res == i)[0])]
            out.append(cl)
        out = [x[2] for x in sorted([(len(x), i, x) for (i, x) in enumerate(out)], reverse = True)]


    # End time count and report
    end = time.time()
    elapsed_time = end - start
    print('Clustering time: ' + time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    
    # Return cluster results
    return out

In [13]:
cl = mf.clusmidf(smidft)

IndexError: tuple index out of range

In [82]:
smidft.head()

,smiles,id
0,COc1cc(Cl)cc2c1OC(C)C(=O)N(c1c(F)cccc1F)C2,s1
1,COc1ccc(N2CC(C(=O)Oc3cc(C)ccc3C)CC2=O)cc1,s2
2,CC1CCC(C(N)=O)CN1C(=O)Nc1cnn(C(C)(C)C)c1,s3
3,CN(Cc1nc2ccccc2c(=O)[nH]1)C(=O)c1cn2ccccc2n1,s4
4,CC(C)c1noc(C(C)N2CCN(CC(=O)N(C)C)CC2)n1,s5


In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import math as m

In [5]:
d = pd.DataFrame({'clid':range(1, len(cl)+1), 'n':map(len, cl)})
d.head()

,clid,n
0,1,889
1,2,215
2,3,495
3,4,148
4,5,454


In [ ]:
d.plot.bar()

In [11]:
plt.scatter(map(m.log, d['clid']), map(m.log, d['n']))

NameError: name 'plt' is not defined

[0.0,
 0.6931471805599453,
 1.0986122886681098,
 1.3862943611198906,
 1.6094379124341003,
 1.791759469228055,
 1.9459101490553132,
 2.0794415416798357,
 2.1972245773362196,
 2.302585092994046,
 2.3978952727983707,
 2.4849066497880004,
 2.5649493574615367,
 2.6390573296152584,
 2.70805020110221,
 2.772588722239781,
 2.833213344056216,
 2.8903717578961645,
 2.9444389791664403,
 2.995732273553991,
 3.044522437723423,
 3.091042453358316,
 3.1354942159291497,
 3.1780538303479458,
 3.2188758248682006,
 3.258096538021482,
 3.295836866004329,
 3.332204510175204,
 3.367295829986474,
 3.4011973816621555,
 3.4339872044851463,
 3.4657359027997265,
 3.4965075614664802,
 3.5263605246161616,
 3.5553480614894135,
 3.58351893845611,
 3.6109179126442243,
 3.6375861597263857,
 3.6635616461296463,
 3.6888794541139363,
 3.713572066704308,
 3.7376696182833684,
 3.7612001156935624,
 3.784189633918261,
 3.8066624897703196,
 3.828641396489095,
 3.8501476017100584,
 3.871201010907891,
 3.8918202981106265,
 3.91

In [10]:
d.head()

NameError: name 'd' is not defined

In [13]:
# Import myfunc at cix folder
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../cix')
import myfuncs as mf
import pandas as pd
from rdkit.Chem import rdBase, RDConfig
from rdkit import Chem
rdBase.DisableLog('rdApp.*') # To make rdkit silent
from rdkit.Chem import PandasTools as pt
from rdkit.Chem import Descriptors
import numpy as np
import chemfp
import csv
import time
import os